In [1]:
import cv2
import numpy as np
from keras.models import load_model
import matplotlib.pyplot as plt
from utils import stackImages

In [2]:
def get_contour(img):
    biggest=np.array([])
    maxArea=0
    contours, hierarchy = cv2.findContours(img,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE) 
    for cnt in contours:
        area=cv2.contourArea(cnt)
        if (area>5000):
            #cv2.drawContours(img_result,cnt,-1,(255,0,0),5)
            peri=cv2.arcLength(cnt,True)
            approx=cv2.approxPolyDP(cnt,0.09*peri,True)      ## return coordinates of points in contour 
            if area>maxArea and len(approx)==4:
                biggest=approx
                maxArea=area
            x,y,w,h=cv2.boundingRect(approx)                 ## bound retangle accros contour or object 
            #cv2.rectangle(img_contour,(x,y),(w,h),(255,0,255),4)
        #cv2.drawContours(img_contour,cnt,-1,(255,0,0),5)
    cv2.drawContours(img_result,biggest,-1,(255,0,255),30)
    return biggest

In [3]:
def image_class(img):
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img_blur=cv2.GaussianBlur(gray,(7,7),0)
    img_canny=cv2.Canny(img_blur,100,100)            # Canny edge detector
    kernal=np.ones((5,5),dtype=np.uint8)                           ## thicknned the edge by increase iteration using dialation
    img_dialated=cv2.dilate(img_canny,kernal,iterations=2)
    img_erode=cv2.erode(img_dialated,kernal,iterations=2)             ## thinned the edge by increase iteration using erodation
    get_contour(img_dialated)
    return img_blur,img_canny,img_dialated,img_erode

In [4]:
def reorder(myPoints):
    myPoints = myPoints.reshape((4, 2))
    myPointsNew = np.zeros((4, 1, 2), dtype=np.int32)
    add = myPoints.sum(1)
    myPointsNew[0] = myPoints[np.argmin(add)]
    myPointsNew[3] =myPoints[np.argmax(add)]
    diff = np.diff(myPoints, axis=1)
    myPointsNew[1] =myPoints[np.argmin(diff)]
    myPointsNew[2] = myPoints[np.argmax(diff)]
    return myPointsNew

In [5]:
def findcolor(img,color):
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    imgHSV=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    lower=np.array(color[0:3])
    upper=np.array(color[3:6])
    mask=cv2.inRange(imgHSV,lower,upper)
    biggest=get_contour(mask)
    imgwarpcolored=np.zeros_like(gray)
    if biggest.size!=0:
        biggest=reorder(biggest)
        pts1=np.float32(biggest)
        pts2=np.float32([[0,0],[widthImg,0],[0,heightImg],[widthImg,heightImg]])
        matrix=cv2.getPerspectiveTransform(pts1,pts2)
        imgwarpcolored=cv2.warpPerspective(img,matrix,(widthImg,heightImg))
        imgwarpcolored=cv2.cvtColor(imgwarpcolored,cv2.COLOR_BGR2GRAY)
    return imgwarpcolored